In [1]:
import sys

In [2]:
sys.path.insert(1, '../')

In [3]:
from utils.dataset import EHRDataset
from model.tokenizer import EHRTokenizer
import pytorch_pretrained_bert as Bert
from torch.utils.data import DataLoader
from model.model import *
from utils.config import *
from utils.optimizer import adam
from model.trainers import TrainerMLM
import pytorch_lightning as pl
from sklearn.model_selection import KFold
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
#from pytorch_lightning.strategies import DeepSpeedStrategy
warnings.filterwarnings('ignore')

from utils.vocabulary import *

In [14]:
#path = 'C:/Users/Johan/Documents/Skola/MasterThesis/Master-thesis/pre-processing/combined-csv-files.csv'
path_patients = '../data/datasets/readmission_data_synthea'
#path_prior = '../data/datasets/prior_table'

In [15]:
prior1 = pd.read_parquet('../data/datasets/prior_table_1')
prior2 = pd.read_parquet('../data/datasets/prior_table_2')

In [16]:
priordataset = pd.concat([prior1, prior2])
data = pd.read_parquet(path_patients)
dataset = data.merge(priordataset, on='subject_id', how='inner')

In [18]:
global_params = {
    'max_seq_len': 32,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 32,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [19]:
dataset['subject_id'] = dataset['subject_id'].apply(lambda x: x.replace('-', ''))

In [20]:
#diagnos_codes = dataset['diagnos_code'].tolist()
#med_codes = dataset['medication_code'].tolist()
#diagnos_codes.extend(med_codes)
#file_path = '../data/vocabularies/Synthea/snomedrxnorm'
#write_codes_to_file(diagnos_codes, file_path)

In [21]:
#ages = dataset['age'].tolist()
#file_path = '../data/vocabularies/Synthea/age'
#write_age_to_file(ages, file_path)

In [22]:
files = {'code':'../data/vocabularies/Synthea/snomedrxnorm.npy',
         'age':'../data/vocabularies/Synthea/age.npy'
        }
tokenizer = EHRTokenizer(task='MLM', filenames=files)

In [23]:
len(tokenizer.getVoc('code').keys())

433

In [24]:
len(tokenizer.getVoc('age').keys())

113

In [25]:
#Shallow
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 4, # number of multi-head attention layers required
    'num_attention_heads': 6, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
    'use_prior':True,
}

In [26]:
conf = BertConfig(model_config)

In [27]:
tensorboarddir = '../logs/'

In [28]:
#kfold=KFold(n_splits=5,shuffle=True)

In [29]:
trainer = pl.Trainer(
            max_epochs=5, 
            gpus=-1,
            logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.TQDMProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [30]:
trainset, testset = train_test_split(dataset, test_size=0.2)

In [31]:
traind = EHRDataset(trainset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)
testd = EHRDataset(testset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)

In [32]:
trainloader = torch.utils.data.DataLoader(traind, batch_size=32, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testd, batch_size=32, shuffle=True, num_workers=4)

In [34]:
model = BertForMaskedLM(conf) #BertForMaskedLM(conf)
params = list(model.named_parameters())
optim = adam(params, optim_param)

t_total value of -1 results in schedule not being applied


In [35]:
patienttrajectory = TrainerMLM(model, optim, optim_param, 0.1)

trainer.fit(
    patienttrajectory, 
    train_dataloaders=trainloader,
);

predictions = trainer.predict(patienttrajectory, dataloaders=testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 13802it [00:00, ?it/s]

In [36]:
avg_acc = sum([ stats['precision'] for stats in predictions ]) / len(predictions)
avg_acc*100

81.96752159452974

In [37]:
PATH = '../saved_models/MLM/model_with_prior_82test'
torch.save(model.state_dict(), PATH)

In [ ]:
# This is for the crossval, might not be runnable since we have done some debugging

'''
for fold,(train_idx,test_idx) in enumerate(kfold.split(data)):
    print('------------fold no---------{}----------------------'.format(fold))
    
    trainer = pl.Trainer(
            max_epochs=5, 
            gpus=-1,
            logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.progress.TQDMProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=train_params['batch_size'], sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=train_params['batch_size'], sampler=test_subsampler)
    
    model = BertForMaskedLM(conf) #BertForMaskedLM(conf)
    params = list(model.named_parameters())
    optim = adam(params, optim_param)
    
    patienttrajectory = PatientTrajectoryPredictor(model, optim, optim_param, metrics=True)
    
    trainer.fit(
        patienttrajectory, 
        train_dataloader=trainloader,
    );
  
    predictions = trainer.predict(patienttrajectory, dataloaders=testloader)
    avg_acc = sum([ stats['Precision'] for stats in predictions ]) / len(predictions)
    
    if avg_acc > best_test:
        PATH = '../saved_models/Shallow_unsuffled{}'.format(fold)
        torch.save(model.state_dict(), PATH)
        best_test = avg_acc
        
    print("Average Test precision for fold {}: {} ".format(fold, avg_acc))
'''

In [ ]:
PATH = '../saved_models/Shallow_unsuffled{}'.format(fold)
torch.save(model.state_dict(), PATH)

In [ ]:
avg_acc

In [ ]:
np.mean(allacc)

### Testing with BEHRTs training approach (without pytorch lightning)

In [ ]:
#import sklearn.metrics as skm

In [ ]:
'''
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    
   # print("Truelabel:")
   # print(truelabel)
    
   # print("Output:")
   # print(outs)
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision
'''

In [ ]:
#conf = BertConfig(model_config)
#model = BertForMaskedLM(conf) 
#model = model.to('cuda')

In [ ]:
#params = list(model.named_parameters())
#optim = adam(params, optim_param)

In [ ]:
'''
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt= 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt +=1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, gender_ids, input_ids, posi_ids, segment_ids, attMask, masked_label, _ = batch
        #print()
        loss, pred, label = model(input_ids, age_ids, gender_ids, segment_ids, posi_ids,attention_mask=attMask, labels=masked_label)
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 200==0:
            print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss/2000, cal_acc(label, pred), time.time()-start))
            temp_loss = 0
            start = time.time()
            
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

   #print("** ** * Saving fine - tuned model ** ** * ")
    #model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    #create_folder(file_config['model_path'])
    #output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    #torch.save(model_to_save.state_dict(), output_model_file)
        
    cost = time.time() - start
    return tr_loss, cost
    
'''

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
trainset = EHRDataset(trainset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)
trainload = DataLoader(dataset=trainset, batch_size=train_params['batch_size'], shuffle=True, num_workers=2)

In [ ]:
import time

In [ ]:
for e in range(100):
    loss, timecost = train(e, trainload)
    loss = loss / len(trainload)
    
    print("Loss after epoch {}: {}".format(e, loss))
    
    